In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

Table of Contents
=================

*   [1  Data loading](#Data-loading)
    *   [1.1  Read collection](#Read-collection)
*   [2  Check changes](#Check-changes)
    *   [2.1  Load previous data](#Load-previous-data)
    *   [2.2  Generate changelogs](#Generate-changelogs)
    *   [2.3  Save data](#Save-data)
*   [3  Data visualization](#Data-visualization)
    *   [3.1  Full data](#Full-data)
    *   [3.2  Card types](#Card-types)
    *   [3.3  Monsters](#Monsters)
        *   [3.3.1  Attributes](#Attributes)
        *   [3.3.2  Primary types](#Primary-types)
            *   [3.3.2.1  Has effect discrimination](#Has-effect-discrimination)
            *   [3.3.2.2  Is pendulum discrimination](#Is-pendulum-discrimination)
            *   [3.3.2.3  By attribute](#By-attribute)
        *   [3.3.3  Secondary types](#Secondary-types)
            *   [3.3.3.1  By attribute](#By-attribute)
            *   [3.3.3.2  By secondary type](#By-secondary-type)
        *   [3.3.4  Monster types](#Monster-types)
            *   [3.3.4.1  By Attribute](#By-Attribute)
            *   [3.3.4.2  By primary type](#By-primary-type)
            *   [3.3.4.3  By secondary type](#By-secondary-type)
        *   [3.3.5  ATK](#ATK)
        *   [3.3.6  DEF](#DEF)
        *   [3.3.7  Level/Rank](#Level/Rank)
            *   [3.3.7.1  ATK statistics](#ATK-statistics)
            *   [3.3.7.2  DEF statistics](#DEF-statistics)
        *   [3.3.8  Pendulum scale](#Pendulum-scale)
            *   [3.3.8.1  ATK statistics](#ATK-statistics)
            *   [3.3.8.2  DEF statistics](#DEF-statistics)
            *   [3.3.8.3  Level/Rank statistics](#Level/Rank-statistics)
        *   [3.3.9  Link](#Link)
            *   [3.3.9.1  ATK statistics](#ATK-statistics)
        *   [3.3.10  Link Arrows](#Link-Arrows)
            *   [3.3.10.1  By combination](#By-combination)
            *   [3.3.10.2  By unique](#By-unique)
            *   [3.3.10.3  By link](#By-link)
    *   [3.4  Spell & Trap](#Spell-&-Trap)
        *   [3.4.1  Properties](#Properties)
    *   [3.5  Effect type](#Effect-type)
        *   [3.5.1  Card type discrimination](#Card-type-discrimination)
    *   [3.6  Archseries](#Archseries)
        *   [3.6.1  By card type](#By-card-type)
        *   [3.6.2  By primary type](#By-primary-type)
        *   [3.6.3  By secondary type](#By-secondary-type)
        *   [3.6.4  By monster type](#By-monster-type)
        *   [3.6.5  By property](#By-property)
    *   [3.7  Artworks](#Artworks)
        *   [3.7.1  By card type](#By-card-type)
        *   [3.7.2  By primary type](#By-primary-type)
    *   [3.8  Errata](#Errata)
        *   [3.8.1  By card type](#By-card-type)
        *   [3.8.2  By primary type](#By-primary-type)
        *   [3.8.3  By artwork](#By-artwork)
    *   [3.9  TCG & OCG status](#TCG-&-OCG-status)
        *   [3.9.1  TGC status](#TGC-status)
            *   [3.9.1.1  By card type](#By-card-type)
            *   [3.9.1.2  By monster type](#By-monster-type)
            *   [3.9.1.3  By archseries](#By-archseries)
        *   [3.9.2  OCG status](#OCG-status)
            *   [3.9.2.1  By card type](#By-card-type)
            *   [3.9.2.2  By monster type](#By-monster-type)
            *   [3.9.2.3  By archseries](#By-archseries)
        *   [3.9.3  TCG vs. OCG status](#TCG-vs.-OCG-status)
*   [4  Epilogue](#Epilogue)
    *   [4.1  HTML export](#HTML-export)
<!-- *   [4.2  Git](#Git) -->

# Data loading

In [ ]:
_ = git.ensure_repo()

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

In [ ]:
def find_cards(collections: List[pd.DataFrame] | pd.DataFrame, merge_data=False) -> pd.DataFrame:
    """
    Process a DataFrame with card names, numbers and/or passwords and return a DataFrame with the card names, quantities and, optionally, additional card data merged from database.

    Args:
        collections (List[pd.DataFrame] | pd.DataFrame): DataFrame or list of DataFrames with card names, numbers and/or passwords.

    Returns:
        ((List[pd.DataFrame] | pd.DataFrame): DataFrame or list of DataFrames with card names, quantities and, optionally, additional card data merged from database.
    """
    if not isinstance(collections, list):
        collections = [collections]

    card_df, _ = load_latest_data(name_pattern="cards")
    if any(
        [
            "Card number" in collection_df and not collection_df[collection_df["Name"].isna()]["Card number"].empty
            for collection_df in collections
        ]
    ):
        set_lists_df, _ = load_latest_data(name_pattern="sets")

    for i, collection_df in enumerate(collections):
        collection_df = collection_df.dropna(how="all", axis=1).dropna(how="all", axis=0)
        collection_df = collection_df.assign(match=np.nan).astype(object)

        if "Card number" in collection_df and not collection_df["Card number"].dropna().empty:
            number_keys = collection_df["Card number"].dropna().str.upper().str.strip()

            list_keys = set_lists_df["Card number"].dropna().str.upper().str.strip()
            list_values = set_lists_df.loc[list_keys.index]["Name"]
            key_name_dict = dict(zip(list_keys, list_values))

            missing = number_keys[~number_keys.isin(list_keys)]
            if missing.count() > 0:
                print(
                    "\nUnable to find the following card(s) by number:\n ⏺",
                    "\n ⏺ ".join(missing.sort_values().unique()),
                )

            matches = number_keys.map(lambda x: key_name_dict.get(x, x))
            collection_df.loc[matches.index, "match"] = matches
            collection_df.drop("Card number", axis=1, inplace=True)

        if "Password" in collection_df and not collection_df[collection_df["match"].isna()]["Password"].empty:
            password_keys = collection_df[collection_df["match"].isna()]["Password"].dropna().astype(int)
            list_keys = card_df["Password"].dropna().astype(int)
            list_values = card_df.loc[list_keys.index]["Name"]
            key_name_dict = dict(zip(list_keys, list_values))

            missing = password_keys[~password_keys.isin(list_keys)]
            if missing.count() > 0:
                print(
                    "\nUnable to find the following card(s) by password:\n ⏺",
                    "\n ⏺ ".join(missing.sort_values().unique().astype(str)),
                )

            matches = password_keys.map(lambda x: key_name_dict.get(x, x))
            collection_df.loc[matches.index, "match"] = matches
            collection_df.drop("Password", axis=1, inplace=True)

        if "Name" in collection_df and not collection_df[collection_df["match"].isna()]["Name"].empty:
            name_keys = collection_df["Name"].dropna().str.lower().str.strip()
            list_keys = card_df["Name"].str.lower().str.strip()
            key_name_dict = dict(zip(list_keys, card_df["Name"]))

            missing = collection_df["Name"].dropna()[~name_keys.isin(list_keys)]
            if missing.count() > 0:
                print("\nUnable to find the following card(s) by name:\n ⏺", "\n ⏺ ".join(missing.sort_values().unique()))

            # Map the "Name" column from list_df to collection_df based on the keys
            matches = name_keys.map(lambda x: key_name_dict.get(x, x))
            collection_df.loc[matches.index, "match"] = matches
            collection_df.drop("Name", axis=1, inplace=True)

        collection_df = collection_df.rename(columns={"match": "Name"})
        columns = list(collection_df.columns.difference(["Count"]))
        collection_df = collection_df.dropna(how="all", axis=1).dropna(how="all", axis=0)
        collection_df = collection_df.groupby(columns, dropna=False).sum().reset_index()
        if merge_data:
            collection_df = collection_df.merge(card_df.drop_duplicates(subset="Name"), on="Name", how="left")
        collections[i] = collection_df

    print("\nCollection data processed.")
    if len(collections) == 1:
        return collections[0]
    else:
        return collections

In [ ]:
# Process the deck data frame
deck_df = find_cards(deck_df, merge_data=True)

In [ ]:
# Get latest file if exist
previous_deck_df, previousdeck_ts = load_latest_data("deck")

if previous_deck_df is not None:
    previous_deck_df = previous_deck_df.astype(
        deck_df[previous_deck_df.columns.intersection(deck_df.columns)].dtypes.to_dict()
    )

In [ ]:
if previous_deck_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_deck_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_deck_df,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

In [ ]:
deck_df

In [ ]:
# Other

# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is not None:
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

In [ ]:
collection_df